In [1]:
import sys
import pathlib

sys.path.append("..")

In [2]:
import mrob 
import numpy as np
import open3d as o3d 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import voxel_slam

## I/O

In [4]:
def parse_trajectories_float(input_path, ts_multiplier=1e9):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            line_tokens = line.strip('\n').split()
            
            timestamp = float(line_tokens[0]) * ts_multiplier
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts


def trajectory_to_se3(trajectory):
    t, Q = trajectory[:3], trajectory[3:]
    R = mrob.geometry.SO3(mrob.geometry.quat_to_so3(Q))
    return mrob.geometry.SE3(R, t)

def read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=0, number_of_clouds=-1, center_distance_threshold=1.5):
    poses = []
    clouds = []

    lidar_so3 = mrob.geometry.SO3(mrob.geometry.quat_to_so3(np.asarray([ 0.7071068, -0.7071068, 0, 0 ])))
    lidar_t = np.asarray([ -0.001, -0.00855, 0.055 ])   
    imu_to_lidar_se3 = mrob.geometry.SE3(lidar_so3, lidar_t).T()

    for ts in sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]:
        imu_pose = trajectory_to_se3(ts_to_quat[ts]).T()

        pose = imu_pose @ imu_to_lidar_se3
        cloud = o3d.io.read_point_cloud(str(ts_to_depth_path[ts]))
        cloud_points = np.asarray(cloud.points)
        f = np.where(np.linalg.norm(cloud_points, axis=1) > center_distance_threshold)[0] 

        cloud.points = o3d.utility.Vector3dVector(cloud_points[f])

        cloud.paint_uniform_color([0.0, 0.0, 0.0])
        
        poses.append(pose)
        clouds.append(cloud)

    return clouds, poses

In [5]:
clouds_path = "/home/ach/Desktop/datasets/hilti/out2"
poses_path = "/home/ach/Desktop/datasets/hilti/exp14_basement_2_imu.txt"

In [6]:
ts_multiplier = 1 / 1e9
ts_to_depth_path = {float(x.stem) * ts_multiplier : x for x in pathlib.Path(clouds_path).iterdir()}
ts_to_quat = parse_trajectories_float(poses_path, ts_multiplier)

In [7]:
clouds, poses = read_hilti_sequence(ts_to_quat, ts_to_depth_path, start_of_sequence=300, number_of_clouds=30, center_distance_threshold=3)

## Optimization

In [8]:
config = voxel_slam.PipelineConfig(voxel_size=2.0, 
                                   ransac_distance_threshold=0.1, 
                                   filter_cosine_distance_threshold=0.2,
                                   backend_verbose=True)

In [9]:
voxel_filter = voxel_slam.NormalsFilter(config.filter_cosine_distance_threshold)
voxel_filter.set_next(voxel_slam.PlaneDistanceFilter()).set_next(voxel_slam.EmptyVoxelsFilter());

In [10]:
pipeline = voxel_slam.VoxelSLAMPipeline(
    feature_filter=voxel_filter,
    optimization_backend=voxel_slam.BaregBackend,
    config=config
)

In [11]:
(opt_clouds, color_to_voxel), opt_poses = pipeline.process(clouds, poses)

FGraph initial error: 72049.05296298438
Iteratios to converge: 52
Chi2: 49668.25845849754
Time (init): 0.015355414


In [ ]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(opt_clouds, opt_poses)])